# Compute the atom centered Coulomb Matrix representation

To install QML library and compare rascal results with it
+ pip install qml ase

To install rascal:
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

In [3]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [4]:
%matplotlib inline
from matplotlib import pyplot as plt

In [5]:
import qml
from ase.io import read
from ase.visualize import view
import numpy as np
import sys
from ase.build import make_supercell

In [12]:
p = '/home/markus/repos/review/librascal/'
sys.path.insert(0, p + 'build2/')
print (p)

print(sys.path)

/home/markus/repos/review/librascal/
['/home/markus/repos/review/librascal/build2/', '/home/markus/repos/review/librascal/build2/', '/home/markus/repos/review/librascal/build2/', '', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/home/markus/.local/lib/python3.6/site-packages', '/home/markus/.local/lib/python3.6/site-packages/AtomDigit-0.1-py3.6.egg', '/home/markus/.local/lib/python3.6/site-packages/ase-3.16.1b1-py3.6.egg', '/home/markus/.local/lib/python3.6/site-packages/Flask-1.0.2-py3.6.egg', '/home/markus/.local/lib/python3.6/site-packages/itsdangerous-0.24-py3.6.egg', '/home/markus/.local/lib/python3.6/site-packages/click-6.7-py3.6.egg', '/home/markus/.local/lib/python3.6/site-packages/subprocess32-3.5.1-py3.6.egg', '/home/markus/.local/lib/python3.6/site-packages/backports.functools_lru_cache-1.5-py3.6.egg', '/usr/local/lib/python3.6/dist-packages', '/usr/local/lib/python3.6/dist-packages/pep8-1.7.1-py3.6.egg', '/usr/local/lib/python3.6/dist-pa

In [13]:
from rascal.representation import SortedCoulombMatrix

In [14]:
# load a small subset of structures from QM9
frames = read(p +'tests/reference_data/dft-smiles_500.xyz',':')
cutoff = 3.
rep = SortedCoulombMatrix(cutoff,sort='rownorm')

In [15]:
new_frames = []
aa = []
for frame in frames:
    new_frames.append(make_supercell(frame,np.eye(3)*3))
print(np.max(list(map(len,new_frames))))

729


In [16]:
# have a look at them
#view(frames)

In [17]:
# Compute the sorted coulomb matrices for the list of structures
%timeit -n 3 -r 3 features = rep.transform(new_frames)

1.33 s ± 17.9 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [18]:
# extract the feature matrix to compare with 
features = rep.transform(new_frames)
test = features.get_feature_matrix().T

In [19]:
features.shape()

TypeError: 'tuple' object is not callable

In [ ]:
# To get the coulomb matrices from QML
def get_coulomb_ref(frame,size,cutoff,flavour):
    from qml.representations import generate_atomic_coulomb_matrix
    
    nuclear_charges = frame.get_atomic_numbers()
    coordinates = frame.get_positions()
    
    
    cm = generate_atomic_coulomb_matrix(nuclear_charges, coordinates, size = size, sorting = flavour,
    central_cutoff = cutoff, central_decay = -1, interaction_cutoff = 1e6, interaction_decay = -1,indices = None)
    
    return cm
def get_coulomb_refs(frames,size,cutoff,flavour='distance'):  
    cms = []
    for frame in frames:
        cms.append(get_coulomb_ref(frame,size,cutoff,flavour))
    return np.vstack(cms)

In [ ]:
# Compare with reference
%timeit -n 3 -r 3 ref = get_coulomb_refs(new_frames,rep.size,cutoff,flavour='row-norm')
ref = get_coulomb_refs(new_frames,rep.size,cutoff,flavour='row-norm')
np.allclose(test,ref)

In [ ]:
from rascal.utils import fps

In [ ]:
sparse_indices,sparse_minmax_d2 = fps(test,Nselect=500,starting_index=None,flavour='simple')

In [ ]:
plt.plot(sparse_minmax_d2)